In [8]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from tkinter import filedialog
from tkintertable import TableCanvas, TableModel
import json
import os
import re

# --------------------------- VARIÁVEIS GLOBAIS (comentários em PT) --------------------------- #
last_path = ""   # Armazena o último caminho utilizado
last_dimensions = {"width": 1550, "height": 700}  # Dimensões padrão da janela

# Agora o dicionário de clients passa a ter, para cada cliente, sub-dicionários com 'account' e 'desk_id'
clients = {
    # Exemplos. Fique à vontade para alterar ou adicionar mais.
    "KAPITALO": {
        "account": "9269353",
        "desk_id": "desk_kap1"
    },
    "KINEA FOF": {
        "account": "9531442",
        "desk_id": "desk_kfof1"
    },
    "Charlie": {
        "account": "54321",
        "desk_id": "desk_charlie"
    },
    "Diana": {
        "account": "98765",
        "desk_id": "desk_diana"
    },
    "Matheus": {
        "account": "9270551",
        "desk_id": "desk_matheus"
    },
    "EDS UJAY": {
        "account": "9689822",
        "desk_id": "celso_1"
    }
}

# Dicionário de caminhos
paths = {}

# --------------------------- FUNÇÕES UTILITÁRIAS --------------------------- #

def copy_to_clipboard(text):
    """
    Copia 'text' para a área de transferência do sistema.
    """
    root.clipboard_clear()
    root.clipboard_append(text)
    root.update()

def copy_error_message():
    """
    Copia a mensagem de erro do label de status para a área de transferência.
    """
    try:
        error_message = status_label.cget("text")
        copy_to_clipboard(error_message)
    except Exception as e:
        status_label.config(text=f"Failed to copy error message: {str(e)}", bg="red", fg="white")

def paste_data():
    """
    Lê dados da área de transferência, separa por linhas e tabs,
    e carrega na tabela (tkintertable). Não utiliza a primeira linha como cabeçalho.
    """
    try:
        clipboard_data = root.clipboard_get()  # Obtém os dados da área de transferência
        rows = clipboard_data.strip().split('\n')  # Divide em linhas
        data = [row.split('\t') for row in rows if row]  # Divide cada linha em colunas

        # Cria nomes genéricos para as colunas
        max_columns = max(len(row) for row in data)  # Determina o número máximo de colunas
        columns = [f"Column {i}" for i in range(max_columns)]  # Define nomes de colunas genéricas

        # Preenche as linhas com valores
        formatted_data = {
            i: {columns[j]: data[i][j] if j < len(data[i]) else '' for j in range(max_columns)}
            for i in range(len(data))
        }

        # Atualiza o modelo da tabela
        model = TableModel()
        model.importDict(formatted_data)

        table.updateModel(model)
        table.redraw()

        status_label.config(text="Data pasted successfully!", bg="green", fg="white")
    except Exception as e:
        status_label.config(text=f"Error: {str(e)}", bg="red", fg="white")


# --------------------------- FUNÇÕES DE PROCESSAMENTO DE DADOS --------------------------- #

def process_di1_symbols(df):
    """
    Processa o DataFrame para identificar e organizar símbolos DI1 em novas colunas.
    Também cria colunas de apregoar, Quantidade apregoar, Executar e Agressividade,
    mantendo a mesma quantidade de colunas que a de símbolos.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo células com símbolos DI1.

    Returns:
        pd.DataFrame: DataFrame atualizado com colunas adicionais para:
            - subtipo (FRA, FlyFRA, SlideFRA, etc.)
            - símbolos DI1
            - lado (BUY ou SELL)
            - quantidades
            - apregoar (True/False, com lógica diferente para 3 e 4 símbolos)
            - Quantidade apregoar (exemplo aqui é None, mas pode adaptar)
            - Executar (exemplo como "N/A", mas pode adaptar)
            - Agressividade (preenchido com 100 em todas as células)
    """

    # Padrão de vencimentos DI1 (DI1 seguido de uma letra e dois dígitos ou apenas uma letra seguida de dois dígitos)
    pattern = r"(DI1[A-Z]\d{2}|[A-Z]\d{2})"

    # Padrão para identificar lado (BUY ou SELL)
    side_pattern = r"\b(S|B|V|C|Sell|Buy|Compra|Venda)\b"

    # Padrão para identificar números inteiros ou valores como "100/100"
    quantity_pattern = r"\b(\d+)(?:/\d+)?\b"

    # Novas colunas
    subtipo = []
    lado = []

    # Dicionários para símbolos e quantidades
    simbolos_dict = {}
    quantity_dict = {}

    # Dicionários para as novas colunas solicitadas
    apregoar_dict = {}
    quantidade_apregoar_dict = {}
    executar_dict = {}
    agressividade_dict = {}

    # Iterar sobre cada linha
    for _, row in df.iterrows():
        found = []
        side_found = ""
        quantities = []

        # Checar cada célula da linha
        for cell in row:
            if isinstance(cell, str):
                matches = re.findall(pattern, cell)
                # Adicionar prefixo DI1 caso esteja ausente
                matches = [f"DI1{m}" if not m.startswith("DI1") else m for m in matches]
                found.extend(matches)

                # Procurar lado (BUY ou SELL)
                side_match = re.search(side_pattern, cell, re.IGNORECASE)
                if side_match:
                    side_value = side_match.group(0).lower()
                    if side_value in ["s", "sell", "v", "venda"]:
                        side_found = "SELL"
                    elif side_value in ["b", "buy", "c", "compra"]:
                        side_found = "BUY"

                # Procurar números inteiros ou a parte antes de "/" para quantidade
                quantity_matches = re.findall(quantity_pattern, cell)
                quantities.extend(map(int, quantity_matches))

        # Remover duplicados mantendo a ordem
        found = list(dict.fromkeys(found))

        # Adicionar subtipo baseado na quantidade de aparições
        if len(found) == 2:
            subtipo.append("FRA")
        elif len(found) == 3:
            subtipo.append("FlyFRA")
        elif len(found) == 4:
            subtipo.append("SlideFRA")
        else:
            subtipo.append(f"Apareceu {len(found)} vez(es)")

        # Adicionar lado encontrado ou "N/A"
        lado.append(side_found if side_found else "N/A")

        # -------------------------------------------------------------------------
        # Preencher colunas de símbolos e quantidades, como no seu código original
        # -------------------------------------------------------------------------
        for i in range(len(found)):
            simbolos_dict.setdefault(f"Simbolo {i+1}", []).append(found[i])

        for i in range(len(found)):
            quantity_dict.setdefault(f"Quantidade {i+1}", []).append(None)

        if quantities:
            # Atribuir a quantidade apenas à última coluna de quantidade correspondente
            # ao número de símbolos
            for i in range(len(found)):
                if i == len(found) - 1:
                    quantity_dict[f"Quantidade {i+1}"][-1] = quantities[0]

        # Preencher colunas restantes com None (caso a linha não tenha tantos símbolos)
        max_len = max(len(simbolos_dict), len(quantity_dict))
        for i in range(len(found), max_len):
            simbolos_dict.setdefault(f"Simbolo {i+1}", []).append(None)
            quantity_dict.setdefault(f"Quantidade {i+1}", []).append(None)

        # -------------------------------------------------------------------------
        # Preencher as novas colunas: apregoar, Quantidade apregoar, Executar e Agressividade
        # -------------------------------------------------------------------------
        # Exemplo de lógica para "apregoar": se 3 vencimentos => [True, True, False],
        # se 4 vencimentos => [True, True, True, False], caso contrário, preencha tudo com False.
        # Adapte conforme as suas regras de negócio.
        if len(found) == 3:
            apregoar_values = ["FALSE", "TRUE", "FALSE"]
        elif len(found) == 4:
            apregoar_values = ["FALSE", "TRUE", "FALSE", "TRUE"]
        else:
            apregoar_values = ["FALSE"] * len(found)

        # Exemplo: quantidade_apregoar = None, executar = "N/A", agressividade = 100 em todas
        quantidade_apregoar_values = [None] * len(found)
        executar_values = ["TRUE"] * len(found)
        agressividade_values = [100] * len(found)

        # Inserir nos dicionários (semelhante ao que foi feito para símbolos)
        for i in range(len(found)):
            apregoar_dict.setdefault(f"apregoar {i+1}", []).append(apregoar_values[i])
            quantidade_apregoar_dict.setdefault(f"Quantidade apregoar {i+1}", []).append(quantidade_apregoar_values[i])
            executar_dict.setdefault(f"Executar {i+1}", []).append(executar_values[i])
            agressividade_dict.setdefault(f"Agressividade {i+1}", []).append(agressividade_values[i])

        # Se a linha tiver menos símbolos do que o max_len, completamos com None (ou False, etc.)
        for i in range(len(found), max_len):
            apregoar_dict.setdefault(f"apregoar {i+1}", []).append(None)
            quantidade_apregoar_dict.setdefault(f"Quantidade apregoar {i+1}", []).append(None)
            executar_dict.setdefault(f"Executar {i+1}", []).append(None)
            agressividade_dict.setdefault(f"Agressividade {i+1}", []).append(None)

    # -------------------------------------------------------------------------
    # Agora adicionamos tudo ao DataFrame.
    # -------------------------------------------------------------------------
    df["Subtipo"] = subtipo
    df["Lado"] = lado

    # Símbolos e Quantidades
    for col_name, values in simbolos_dict.items():
        df[col_name] = values
    for col_name, values in quantity_dict.items():
        df[col_name] = values

    # Novas colunas (apregoar, Qtd apregoar, Executar, Agressividade)
    for col_name, values in apregoar_dict.items():
        df[col_name] = values
    for col_name, values in quantidade_apregoar_dict.items():
        df[col_name] = values
    for col_name, values in executar_dict.items():
        df[col_name] = values
    for col_name, values in agressividade_dict.items():
        df[col_name] = values

    # -------------------------------------------------------------------------
    # Selecionar colunas para manter
    # Vamos manter: ["Client", "Account", "Desk ID", "Subtipo", "Lado"]
    # e qualquer coluna que comece com Simbolo, Quantidade, apregoar, Executar ou Agressividade.
    # -------------------------------------------------------------------------
    columns_to_keep = (
        ["Client", "Account", "Desk ID", "Subtipo", "Lado"] +
        [
            col for col in df.columns
            if col.startswith("Simbolo") or
               col.startswith("Quantidade") or
               col.startswith("apregoar") or
               col.startswith("Executar") or
               col.startswith("Agressividade")
        ]
    )

    # Intersecta com as colunas existentes, para evitar KeyError se algo não existir
    columns_to_keep = [c for c in columns_to_keep if c in df.columns]

    df = df[columns_to_keep]
    return df

def generate_dataframe():
    """
    Verifica se os campos (Client, Account, Path, Desk ID) e a tabela estão preenchidos.
    Se estiver tudo ok, cria um DataFrame pandas a partir da tabela, processa e exibe em uma nova aba.
    """
    try:
        client = picklist_client.get()
        account = entry_account.get()
        desk_id = picklist_desk.get()
        path_value = picklist_path.get()

        if not client or client == "Select a name":
            raise ValueError("The 'Client' field is required.")
        if not account or account == "Enter account number":
            raise ValueError("The 'Account' field is required.")
        if not desk_id or desk_id == "Select a Desk ID":
            raise ValueError("The 'Desk ID' field is required.")
        if not path_value:
            raise ValueError("The 'Path' field is required.")
        if table.model.getRowCount() == 0 or all(all(table.model.getValueAt(row, col) == '' for col in range(table.model.getColumnCount())) for row in range(table.model.getRowCount())):
            raise ValueError("The table is empty.")

        data_values = []
        columns = [table.model.getColumnName(i) for i in range(table.model.getColumnCount())]
        for row_index in range(table.model.getRowCount()):
            row_data = []
            for col_index in range(table.model.getColumnCount()):
                row_data.append(table.model.getValueAt(row_index, col_index))
            data_values.append(row_data)

        df = pd.DataFrame(data_values, columns=columns)

        # Agora processa o DF para identificar símbolos DI1
        df_tranche = process_di1_symbols(df)
        display(df_tranche)

        # Vamos inserir as colunas de Client, Account e Desk ID no DF antes de exibir
        df_tranche.insert(0, "Cliente", client)
        df_tranche.insert(2, "Desk ID", desk_id)

        # Abre uma nova janela exibindo o DataFrame e com botão de download
        display_dataframe_in_new_window(df_tranche, path_value)

        status_label.config(text="DataFrame generated successfully!", bg="green", fg="white")
    except ValueError as ve:
        status_label.config(text=str(ve), bg="red", fg="white")
    except Exception as e:
        status_label.config(text=f"Error: {str(e)}", bg="red", fg="white")

def display_dataframe_in_new_window(df, path_value):
    """
    Cria uma nova janela (Toplevel) para exibir o DataFrame em forma de Treeview
    com scrollbars vertical e horizontal, centralizado na tela.
    Também adiciona um botão para download em Excel na pasta definida.
    """
    top = tk.Toplevel(root)
    top.title("Generated DataFrame Preview")

    # Define o tamanho desejado da nova janela
    width = 900
    height = 500

    # Calcula a posição para centralizar a janela na tela
    x = (root.winfo_screenwidth() // 2) - (width // 2)
    y = (root.winfo_screenheight() // 2) - (height // 2) - 80
    top.geometry(f"{width}x{height}+{x}+{y}")

    # Permite redimensionamento, se desejar
    top.resizable(True, True)

    # Frame para Treeview
    frame_tree = tk.Frame(top)
    frame_tree.pack(fill="both", expand=True)

    # Define as colunas do Treeview
    columns = list(df.columns)
    tree = ttk.Treeview(frame_tree, columns=columns, show="headings")

    # Barra de rolagem vertical
    vsb = ttk.Scrollbar(frame_tree, orient="vertical", command=tree.yview)
    vsb.pack(side="right", fill="y")
    tree.configure(yscrollcommand=vsb.set)

    # Barra de rolagem horizontal
    hsb = ttk.Scrollbar(frame_tree, orient="horizontal", command=tree.xview)
    hsb.pack(side="bottom", fill="x")
    tree.configure(xscrollcommand=hsb.set)

    # Configura cabeçalhos (largura de 120 px, ajustável conforme necessidade)
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, anchor=tk.CENTER, width=120, stretch=True)

    # Insere os dados no Treeview
    for index, row in df.iterrows():
        tree.insert("", "end", values=list(row))

    tree.pack(fill="both", expand=True)

    # Botão para download do Excel
    def download_excel():
        try:
            # Nome do arquivo de saída
            output_file = os.path.join(path_value, "output_dataframe.xlsx")
            df.to_excel(output_file, index=False)
            status_label.config(text=f"Excel file saved at: {output_file}", bg="green", fg="white")
        except Exception as e:
            status_label.config(text=f"Error saving Excel: {str(e)}", bg="red", fg="white")

    download_button = tk.Button(top, text="Download Excel", command=download_excel,
                                bg="#800000", fg="white")
    download_button.pack(pady=5)


# --------------------------- FUNÇÕES DE INTERFACE (UI) --------------------------- #

def clear_table():
    """
    Limpa todos os dados da tabela atual (reinicia a aplicação).
    """
    try:
        root.destroy() # Fecha a janela atual
        main() # Reinicia a aplicação
    except Exception as e:
        error_message = f"Error: {str(e)}"
        print(error_message)
        status_label.config(text=error_message, bg="red", fg="white")

def update_account(event):
    """
    Quando um client é selecionado, atualiza o campo 'Account' e
    carrega a Desk ID (se existir).
    """
    client = picklist_client.get()
    account_value = ""
    desk_value = ""

    if client in clients:
        account_value = clients[client].get("account", "")
        desk_value = clients[client].get("desk_id", "")

    # Atualiza o campo de Account
    entry_account.delete(0, tk.END)
    entry_account.insert(0, account_value)

    # Atualiza o combobox de Desk ID
    picklist_desk.set("")
    picklist_desk['values'] = []
    if desk_value:  
        # Se quiser permitir apenas um desk ID por cliente, setamos diretamente:
        picklist_desk['values'] = [desk_value]
        picklist_desk.set(desk_value)
    else:
        # Caso não exista desk_id, mostra placeholder
        picklist_desk.set("Select a Desk ID")

def update_client(event):
    """
    Quando a entrada de 'Account' perde foco, tenta identificar qual client tem aquela conta.
    Se não achar, mostra 'Account not found'.
    Depois atualiza a combobox de Desk ID.
    """
    account_value = entry_account.get()
    found_client = None
    for k, info in clients.items():
        if info.get("account") == account_value:
            found_client = k
            break

    if found_client:
        picklist_client.set(found_client)
        # Ao atualizar manualmente o picklist_client, chamamos update_account para sincronizar Desk ID
        update_account(None)
    else:
        picklist_client.set("Account not found")
        picklist_desk.set("Select a Desk ID")

def load_json_data():
    """
    Carrega de 'data.json': clients, paths, last_path e last_dimensions.
    Se não existir, cria dicionários padrão.
    """
    global clients, paths, last_path, last_dimensions
    if os.path.exists('data.json'):
        with open('data.json', 'r') as f:
            data = json.load(f)
            # Importante: precisamos verificar se o formato de clients mudou
            # Se no seu caso estiver migrando de um formato antigo, adapte aqui.
            clients = data.get("clients", clients)
            paths = data.get("paths", {})
            last_path = data.get("last_path", "")
            last_dimensions = data.get("last_dimensions", {"width": 1550, "height": 700})
    else:
        # Se não existir, salva o dicionário atual (que já foi definido no início)
        save_json_data()

def save_json_data():
    """
    Salva as variáveis: clients, paths, last_path e last_dimensions em 'data.json'.
    """
    with open('data.json', 'w') as f:
        json.dump({
            "clients": clients,
            "paths": paths,
            "last_path": last_path,
            "last_dimensions": last_dimensions
        }, f)

def update_comboboxes():
    """
    Atualiza as listas de valores das comboboxes: picklist_client e picklist_path.
    """
    picklist_client['values'] = list(clients.keys())
    picklist_path['values'] = list(paths.keys())

def add_path():
    """
    Abre um diálogo para selecionar diretório. Se for escolhido, adiciona em 'paths'
    e define como 'last_path'.
    """
    global last_path
    new_path = filedialog.askdirectory(title="Select a directory")
    if new_path:
        if new_path not in paths:
            paths[new_path] = new_path
        last_path = new_path
        picklist_path.set(new_path)
        save_json_data()
        update_comboboxes()
        status_label.config(text="Path added/selected successfully!", bg="green", fg="white")
    else:
        status_label.config(text="No directory selected.", bg="yellow", fg="black")

def delete_path():
    """
    Deleta o caminho atualmente selecionado na combobox 'picklist_path'.
    Se for igual ao 'last_path', limpa last_path.
    """
    global last_path
    selected_path = picklist_path.get()
    if selected_path in paths:
        del paths[selected_path]
        if last_path == selected_path:
            last_path = ""
        save_json_data()
        update_comboboxes()
        picklist_path.set("")
        status_label.config(text="Path deleted!", bg="green", fg="white")
    else:
        status_label.config(text="Path not found.", bg="red", fg="white")

def on_path_selected(event):
    """
    Quando o usuário seleciona um caminho na combobox 'picklist_path',
    atualiza last_path e salva no JSON.
    """
    global last_path
    chosen_path = picklist_path.get()
    if chosen_path:
        last_path = chosen_path
        save_json_data()
        status_label.config(text=f"New path selected: {chosen_path}", bg="green", fg="white")

def save_window_dimensions(event):
    """
    Salva as dimensões da janela no arquivo JSON ao redimensionar.
    """
    global last_dimensions
    last_dimensions["width"] = root.winfo_width()
    last_dimensions["height"] = root.winfo_height()
    save_json_data()

def add_placeholder(entry, placeholder):
    """
    Adiciona um placeholder ao campo de entrada.
    """
    entry.insert(0, placeholder)
    entry.bind("<FocusIn>", lambda e: clear_placeholder(e, placeholder))
    entry.bind("<FocusOut>", lambda e: restore_placeholder(e, placeholder))

def clear_placeholder(event, placeholder):
    if event.widget.get() == placeholder:
        event.widget.delete(0, tk.END)

def restore_placeholder(event, placeholder):
    if not event.widget.get():
        event.widget.insert(0, placeholder)

# --------------------------- FUNÇÃO PRINCIPAL --------------------------- #

def main():
    global root, table, status_label
    global picklist_client, entry_account, picklist_path, picklist_desk

    load_json_data()

    # Cria a janela principal
    root = tk.Tk()
    root.title("Tranche Importer")

    # Define o ícone (somente no Windows)
    #root.iconbitmap('caminho/para/seu_icone.ico')

    # Dimensões da janela
    window_width = last_dimensions["width"]
    window_height = last_dimensions["height"]

    # Calcula a posição para centralizar a janela na tela
    x = (root.winfo_screenwidth() // 2) - (window_width // 2)
    y = (root.winfo_screenheight() // 2) - (window_height // 2)

    root.geometry(f"{window_width}x{window_height}+{x}+{y}")
    root.minsize(1550, 700)
    root.resizable(True, True)
    root.bind("<Configure>", save_window_dimensions)

    style = ttk.Style()
    style.theme_use('clam')

    # Frame para a tabela
    frame_table = tk.Frame(root)
    frame_table.pack(fill="both", expand=True)

    # Criação de uma tabela inicial vazia
    data = {
        i: {f'Column {j}': '' for j in range(40)}  
        for i in range(35)
    }
    model = TableModel()
    model.importDict(data)
    table = TableCanvas(frame_table, model=model)
    table.show()

    default_font = ("Arial Narrow", 10, "bold")

    # --------------------------- Painel Inferior de Controles --------------------------- #
    control_frame = tk.Frame(root)
    control_frame.pack(side="bottom", fill="x", padx=5)

    # Label Client
    label_client = tk.Label(control_frame, text="Client:", font=default_font)
    label_client.pack(side="left", fill="x", anchor="w", expand=True)

    # Combobox Client
    picklist_client = ttk.Combobox(
        control_frame,
        values=list(clients.keys()),
        font=("Arial Narrow", 14),  # Fonte maior
        width=25  # Largura ajustada
    )
    picklist_client.set("Select a name")
    picklist_client.pack(side="left", fill="x", anchor="w", padx=(0, 5), expand=True)
    picklist_client.bind("<<ComboboxSelected>>", update_account)

    # Label Account
    label_account = tk.Label(control_frame, text="Account:", font=default_font)
    label_account.pack(side="left", fill="x", anchor="w", expand=True)

    # Entry Account
    entry_account = tk.Entry(
        control_frame, 
        font=("Arial Narrow", 14),  # Fonte maior
        width=20,  # Largura ajustada
        borderwidth=1,  # Borda mais espessa
        relief="groove"  # Borda sólida
    )
    entry_account.pack(side="left", fill="x", anchor="w", padx=(0, 5), pady=10, expand=True)  # Espaçamento vertical
    add_placeholder(entry_account, "Enter account number")  # Adiciona placeholder
    entry_account.bind("<FocusOut>", update_client)

    # Label Desk ID
    label_desk = tk.Label(control_frame, text="Desk ID:", font=default_font)
    label_desk.pack(side="left", fill="x", anchor="w", expand=True)

    # Combobox Desk ID
    picklist_desk = ttk.Combobox(
        control_frame,
        values=[],  # Será preenchido dinamicamente
        font=("Arial Narrow", 14),  # Fonte maior
        width=20  # Largura ajustada
    )
    picklist_desk.set("Select a Desk ID")
    picklist_desk.pack(side="left", fill="x", anchor="w", padx=(0, 5), expand=True)

    # Botão Paste Data
    paste_button = tk.Button(control_frame, text="Paste Data", command=paste_data,
                             bg="#800000", fg="white", font=default_font, width=20)
    paste_button.pack(side="left", padx=5, expand=True)

    # Botão Clear Table
    clear_button = tk.Button(control_frame, text="Clear Data", command=clear_table,
                             bg="#800000", fg="white", font=default_font, width=20)
    clear_button.pack(side="left", padx=5, expand=True)

    # Botão Generate DataFrame
    generate_button = tk.Button(control_frame, text="Generate Tranche", command=generate_dataframe,
                                bg="green", fg="white", font=default_font, width=20)
    generate_button.pack(side="left", padx=5, expand=True)


    # --------------------------- Painel Inferior para os Caminhos --------------------------- #
    path_frame = tk.Frame(root)
    path_frame.pack(side="bottom", fill="x", padx=10, pady=10)

    picklist_path = ttk.Combobox(
        path_frame,
        values=list(paths.keys()),
        font=("Arial Narrow", 14),  # Fonte maior
        width=30  # Largura ajustada
    )
    picklist_path.pack(side="left", padx=5, fill="x", expand=True)

    if last_path:
        picklist_path.set(last_path)
    picklist_path.bind("<<ComboboxSelected>>", on_path_selected)

    add_path_button = tk.Button(path_frame, text="Add Path", command=add_path,
                                bg="#800000", fg="white", font=default_font, width=15)
    add_path_button.pack(side="left", padx=5)

    delete_path_button = tk.Button(path_frame, text="Delete Path", command=delete_path,
                                   bg="#800000", fg="white", font=default_font, width=15)
    delete_path_button.pack(side="left", padx=5)

    # --------------------------- Label de Status e Botão Copy Message --------------------------- #
    status_frame = tk.Frame(root)
    status_frame.pack(side="bottom", fill="x", padx=5, pady=5)

    status_label = tk.Label(status_frame, text="", font=default_font, bg="lightgray", anchor="center")
    status_label.pack(side="left", fill="x", expand=True, padx=10)

    copy_button = tk.Button(status_frame, text="Copy Message", command=copy_error_message,
                             bg="#800000", fg="white", font=default_font, width=20)
    copy_button.pack(side="right", padx=10)

    root.mainloop()

if __name__ == "__main__":
    main()

,Subtipo,Lado,Simbolo 1,Simbolo 2,Simbolo 3,Quantidade 1,Quantidade 2,Quantidade 3,apregoar 1,apregoar 2,apregoar 3,Quantidade apregoar 1,Quantidade apregoar 2,Quantidade apregoar 3,Executar 1,Executar 2,Executar 3,Agressividade 1,Agressividade 2,Agressividade 3
0,FlyFRA,BUY,DI1J26,DI1N26,DI1V26,None,None,100,FALSE,TRUE,FALSE,None,None,None,TRUE,TRUE,TRUE,100,100,100
1,FlyFRA,BUY,DI1J26,DI1N26,DI1V26,None,None,100,FALSE,TRUE,FALSE,None,None,None,TRUE,TRUE,TRUE,100,100,100
2,FlyFRA,SELL,DI1J26,DI1N26,DI1V26,None,None,100,FALSE,TRUE,FALSE,None,None,None,TRUE,TRUE,TRUE,100,100,100
3,FlyFRA,SELL,DI1J26,DI1N26,DI1V26,None,None,100,FALSE,TRUE,FALSE,None,None,None,TRUE,TRUE,TRUE,100,100,100
